In [1]:
# imageio
import imageio

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.animation import FuncAnimation
from pathlib import Path
import os
import gc

# Define vehicle dimensions
vehicle_length = 4.18  # meters
vehicle_width = 1.99  # meters

def get_vehicle_corners(x, y, yaw, length, width):
    # Calculate rear center position
    # rear_x = x - (length / 2) * np.cos(np.radians(yaw))
    # rear_y = y - (length / 2) * np.sin(np.radians(yaw))
    
    # Calculate corner points
    corners_x = [x + (length / 2) * np.cos(np.radians(yaw)) - (width / 2) * np.sin(np.radians(yaw)), # Front Right
                 x + (length / 2) * np.cos(np.radians(yaw)) + (width / 2) * np.sin(np.radians(yaw)), # Front Left
                 x - (length / 2) * np.cos(np.radians(yaw)) + (width / 2) * np.sin(np.radians(yaw)), # Rear Left
                 x - (length / 2) * np.cos(np.radians(yaw)) - (width / 2) * np.sin(np.radians(yaw)), # Rear Right
                 x + (length / 2) * np.cos(np.radians(yaw)) - (width / 2) * np.sin(np.radians(yaw))]
    
    corners_y = [y + (length / 2) * np.sin(np.radians(yaw)) + (width / 2) * np.cos(np.radians(yaw)), # Front Right
                 y + (length / 2) * np.sin(np.radians(yaw)) - (width / 2) * np.cos(np.radians(yaw)), # Front Left
                 y - (length / 2) * np.sin(np.radians(yaw)) - (width / 2) * np.cos(np.radians(yaw)), # Rear Left
                 y - (length / 2) * np.sin(np.radians(yaw)) + (width / 2) * np.cos(np.radians(yaw)), # Rear Right
                 y + (length / 2) * np.sin(np.radians(yaw)) + (width / 2) * np.cos(np.radians(yaw))]
    
    return corners_x, corners_y

# Animation function
def animate(i):
    global First_crossed
    ax.clear()
    ax2.clear()

    # Flip the x and y axis limits
    ax.set_ylim(min(min(data.x_FV), min(data.x_MV)) - 10, max(max(data.x_FV), max(data.x_MV)) + 10)
    ax.set_xlim(min(min(data.y_FV), min(data.y_MV)) - 10, max(max(data.y_FV), max(data.y_MV)) + 10)
    
    # Draw lanes as horizontal dashed lines instead of vertical
    for lane_y in [9.9, 13.3, 16.8]:
        ax.axhline(y=lane_y, color='gray', linestyle='--')
    
    # Get vehicle corners for each car and flip x and y coordinates for plotting
    if 'Yaw_FV' in data.columns:
        corners_x_FV, corners_y_FV = get_vehicle_corners(data['x_FV'].iloc[i], data['y_FV'].iloc[i], data['Yaw_FV'].iloc[i], vehicle_length, vehicle_width)
    else:
        corners_x_FV, corners_y_FV = get_vehicle_corners(data['x_FV'].iloc[i], data['y_FV'].iloc[i], -90.29, vehicle_length, vehicle_width)
    corners_x_MV, corners_y_MV = get_vehicle_corners(data['x_MV'].iloc[i],data['y_MV'].iloc[i],  data['Yaw_MV'].iloc[i], vehicle_length, vehicle_width)
    if 'x_LV' in data.columns and 'y_LV' in data.columns:
        corners_x_LV, corners_y_LV = get_vehicle_corners(data['x_LV'].iloc[i],data['y_LV'].iloc[i],  -90.29, vehicle_length, vehicle_width)
        ax.plot(corners_y_LV, corners_x_LV, 'c')  # Flip x and y
    
    # Plot Front Vehicle (FV) and Main Vehicle (MV) with flipped axes
    ax.plot(corners_y_FV, corners_x_FV, 'r')  # Flip x and y
    ax.plot(corners_y_MV, corners_x_MV, 'b')  # Flip x and y
    
    ax.plot([-175, -169], [13.3, 16.8], color='black')
    
    ax.set_title(f"Time: {data['Time'].iloc[i]:.2f}")

    if First_crossed is None:
        for j in range(len(data)):
            corners_x_MV, _ = get_vehicle_corners(data['x_MV'].iloc[j], data['y_MV'].iloc[j], data['Yaw_MV'].iloc[j], vehicle_length, vehicle_width)
            # 전방 왼쪽 꼭지점은 corners_x_MV[1]을 사용하여 접근
            if corners_x_MV[1] < 13.3:
                First_crossed = data['Time'].iloc[j]
                break

    # 두 번째 그래프(Interacting possibility) 설정
    ax2.plot(data['Time'], data['Filtered_Interacting_possibility'], color='#FF8C00')
    if First_crossed is not None:
        ax2.axvline(x=First_crossed, color='blue', linestyle='--', linewidth=2)
    ax2.set_ylim(-0.1, 1.1)  # Interacting possibility의 범위 설정
    ax2.set_xlim(data['Time'].iloc[0]-1, data['Time'].iloc[-1]+1)

    # LeftBlinker_MV가 True인 지점 표시
    blinker_on_data = data[data['LeftBlinker_MV']]
    for time in blinker_on_data['Time']:
        ax2.axvline(x=time, color='lightgreen', alpha=0.5)
    
    # 현재 시점 표시
    current_time = data['Time'].iloc[i]
    ax2.axvline(x=current_time, color='black')

# Define vehicle dimensions
vehicle_length = 4.18  # meters
vehicle_width = 1.99  # meters

START = False
images = []


In [2]:
i = 16
folder_path = Path('./DS_experiment_data/Exp'+str(i)+'/')
csv_files = folder_path.glob('*.csv')


for file_path in csv_files:
    
    del images
    gc.collect()
    SUCCESS = None
    First_crossed = None
    # Read the last row of the current CSV file
    data = pd.read_csv(file_path)

    # Filter possibility
    data['Filtered_Interacting_possibility'] = data['Interacting possibility'].copy()
    data.loc[~((data['x_MV'] > 13.3) & (data['gap_btw_MV'] > - vehicle_length)), 'Filtered_Interacting_possibility'] = np.nan

    if data['y_MV'].iloc[-1] < data['y_FV'].iloc[-1]:
        SUCCESS = True
    else:
        SUCCESS = False

    file_name = os.path.basename(file_path)
    file_name = file_name.replace('.csv','')
    # Setup figure and axis
    
    width = 4*(max(max(data.y_FV), max(data.y_MV)) + 10-min(min(data.y_FV), min(data.y_MV)) + 10)/(max(max(data.x_FV), max(data.x_MV)) + 10 - min(min(data.x_FV), min(data.x_MV)) + 10)

    # Creating animation
    images = []
    for j in range(len(data)):
        fig, axs = plt.subplots(2,1,figsize=(width, 8))
        ax = axs[0]
        ax2 = axs[1]
        animate(j)
        fig.canvas.draw()
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        images.append(image)
        plt.close()
    if SUCCESS:
        imageio.mimsave('./DS_experiment_data/Exp'+str(i)+'/'+file_name+'_success.gif',images, writer='imagemagick' ,loop=0, fps=20)
    else:
        imageio.mimsave('./DS_experiment_data/Exp'+str(i)+'/'+file_name+'_failure.gif',images, writer='imagemagick', loop=0, fps=20)
    print(file_path,'DONE')
    plt.close()  # Prevent extra static plot from displaying


DS_experiment_data\Exp16\10_10_40_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\10_10_50_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\10_10_60_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\10_10_60_merging_scenario_data_1.csv DONE
DS_experiment_data\Exp16\10_20_40_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\10_30_50_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\20_10_40_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\20_10_40_merging_scenario_data_1.csv DONE
DS_experiment_data\Exp16\20_10_50_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\20_10_60_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\20_10_60_merging_scenario_data_1.csv DONE
DS_experiment_data\Exp16\20_20_40_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\20_20_50_merging_scenario_data.csv DONE
DS_experiment_data\Exp16\20_20_50_merging_scenario_data_1.csv DONE
DS_experiment_data\Exp16\20_30_60_merging_scenario_data.csv DONE
DS_experiment_dat